# Projeto 1 - Ciência dos Dados

Nome: João Gabriel Valentim Rocha

Nome: Enzo Dadier Lacks Zamberlan

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

In [76]:
!pip install seaborn
!pip install emoji
!pip install pysinonimos
!pip install nltk
!pip install sklearn

  Created wheel for emoji: filename=emoji-1.5.0-py3-none-any.whl size=187447 sha256=27aa3dbc73cb8d6664c0b6f79daa5b81c789451d7496485d1ca0db2f7db91bbc
  Stored in directory: c:\users\gabri\appdata\local\pip\cache\wheels\e9\d5\46\33c9101a710eb267fbaf8572c96fce5b12702e62bb5fa40592
Successfully built emoji
  Created wheel for pysinonimos: filename=pysinonimos-0.1.1-py3-none-any.whl size=2458 sha256=aea325f593ed2b893c015dcfc5a2445575e1057d512f015876836ee199513126
  Stored in directory: c:\users\gabri\appdata\local\pip\cache\wheels\52\3b\eb\f40f66159b8c2a9b9b25c4e1f91ff7f566b7f2305268adf062
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-py3-none-any.whl size=12552 sha256=51d9858864eb605c304b359b55d897e22242f36e2ecd7c7dd08434a2da29b6bb
  Stored in directory: c:\users\gabri\appdata\local\pip\cache\wheels\d1\d7\61\11b5b370ee487d38b5408ecb7e0257db9107fa622412cbe2ff
  Created wheel for protego: filename=Protego-0.1.16-py3-none-any.whl size=7770 sha256=100548b250ffa97a0ec2a3a995a3db5

In [121]:
!pip install unidecode

___
Carregando algumas bibliotecas:

In [79]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [127]:
import re
import string
import emoji
import nltk
import pysinonimos.sinonimos as sinom
from emoji import UNICODE_EMOJI
import unidecode

In [81]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\gabri\Desktop\Matérias 2 semestre\cdados\P1\Classificador_automatico_de_sentimentos


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [82]:
filename = 'assets/loki.xlsx'

In [141]:
train = pd.read_excel(filename)
train

,Treinamento,Relevancia
0,mood de hj: bolsominion veio falar com loki:\n...,0
1,sdds do loki,1
2,"considerações finais de loki: parabéns marvel,...",1
3,@_namizinhaa @yaluv_ @_girlrainbow versão femi...,0
4,o loki merece todo o amor do mundo ? \n \n si...,1
...,...,...
295,o loki tbm ? percebi que nao sei mais nada sob...,1
296,tô desde as 9 fazendo atividade e o loki destr...,0
297,@torystyliinson @mobiusdaavt pede pra ele te m...,0
298,"bolsonaro tá desesperado. discurso inflado, go...",0


In [84]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevancia
0,comecei loki e logo no primeiro episódio ele s...,1
1,@thylaspirk exatamente \nnão dá cara loki mere...,1
2,pq a marvel falou que loki é irmão se thor sen...,0
3,vou fazer com o loki 🥰🤣 https://t.co/qry6pytktj,1
4,@stawocon loki eu ainda vi dnv ksksksksksksksk...,1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O classificador automático de sentimentos montado tem como critério utilizado a relevância de tweets que remetam a alguma reação ou laço sentimental em relação à série, sendo os mesmos positivos ou não (elogiando algum episódio, comentando valores da série, ou ainda, criticando os mesmos). Neste sentido, postagens que dizem respeito unicamente a aspectos nesse tocante, elogiando ou comentando sobre os atores da obra, como também tweets com vagas menções sobre a série, foram considerados como irrelevantes.

### Procedimentos e configurações iniciais

Para continuarmos, é necessário definir algumas funções básicas que vão nos ajudar:

1) Primeiro, vamos fazer a limpeza dos tweets. Retirando simbolos e pontuações que não serão contabilizadas para nossa análise

2)

In [115]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [156]:
# Passo 1
def limpa_frase(frase):
    # Primeiro, poe todas as palavras com letras minúsculas
    aux = frase.lower()
    
    # Segundo, remove # and @
    aux = re.sub("@[A-Za-z0-9_]+","", aux)
    aux = re.sub("#[A-Za-z0-9_]+","", aux)
    
    # Terceiro, remove links
    aux = re.sub(r"http\S+", "", aux)
    aux = re.sub(r"www.\S+", "", aux)
    
    # Quarto, remove pontuação
    aux = re.sub('[()!?]', ' ', aux)
    aux = re.sub('\[.*?\]',' ', aux)
    
    # Quinto, remove acentos
    #aux = unidecode.unidecode(aux)
    
    # Sexto, remove não alfa-numericos
    #aux = re.sub("[^a-z0-9]"," ", aux)
    return aux

In [152]:
# Passo 2
def tokenize(frase):
    return frase.split()

In [95]:
# Passo 3 
nltk.download('stopwords')
prep = nltk.corpus.stopwords.words('portuguese')
prep.append('')
prep = prep + ['n', 's']

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [153]:
# Passo 4
def no_stop_words(token):
    clear = []
    for element in token:
        if element not in prep:
            clear.append(element)
    return clear

In [157]:
frase_1 = test['Teste'][3]

In [158]:
print(no_stop_words(tokenize(limpa_frase(frase_1))))

['vou', 'fazer', 'loki', '🥰🤣']


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**